# Experiments with variable flows

We saw in the previous tutorials how to run simulations with pre-set networks. Pre-set networks can also be runned with flows defined in a matrix, and evolving during the simulation. In this tutorial, we're going to see how to run this simulations. We will use the `OneCrossroadNetwork` to show this.

In [1]:
from sumo_experiments import Experiment
from sumo_experiments.preset_networks import OneCrossroadNetwork

network = OneCrossroadNetwork()

## Generate variable flows with a matrix

For every pre-set network, three flows functions are implemented : 
- `generate_flows_only_ahead` defines flows where vehicles can only go ahead at intersections.
- `generate_flows_all_directions` defines flows where vehicles can go to every direction at intersections.
- `generate_flows_with_matrix` defines flows following a schema described in a matrix.

The first two functions implements the parameters `stop_generation_time` and `flow_frequency` that define the flow frequency and the simulation step when flows will end. This setting is set __only one time__. It's impossible to change this parameters during the simulation.

As it can be useful to change flows during a simulation, to experiment the adaptation of traffic light behaviour for example, the library implements the `generate_flow_with_matrix` that creates flows evolving over time. Let's see how to use it.

The function works as following. We declare matrix whose first line is the **load vector** for all the network, and the other lines are the **proportion of flows that are using a specific route** (between 0 and 1). Thus, each column of the matrix is the setting for a part of the simulation (a period), a load of vehicle for the network, and the proportion of frequency using each route. The time of a period is defined as a **period time**, and is the same for each period of the simulation.

The routes of a network are defined as couple entry-exit. In the pre-set networks, a vehicle can enter the network from each entry and leave it by any exit (except the one it has entered). For example, the `OneCrossroadNetwork`, which is the simpliest pre-set network, has 4 entry and 4 exit. Then, the total number of routes is 12 (4\*3). For a SquareNetwork with a side length of 5 entries, this number rise to 380.

The order of the routes in the matrix is as following : the first line represents the most north-west entry, and the exit is the first exit clockwise. We then iterate over all exits clockwise. The next entry is the next entry clockwise, and we restart to iterate over all exits from the most north-west. For exemple, for the `OneCrossroadNetwork`, the 12 lines matrix must have the following order (entry / exit):
1) North / East
2) North / South
3) North / West
4) East / North
5) East / South
6) East / Wes
7) South / North
8) South / East
9) South / West
10) West / North
11) West / East
12) West / South

The code below shows an implementation of an Experiment where flows are defined by a load vector and a matrix of coefficient. We define 3 period of time. A period last 300 simulation steps. For the first period, the flows are distributed equally between each route. For the second period, only routes that start by the north entry or the south entry are activated. The repartition of flows between routes is set equally. Then, in the third and last period, we set behaviour as previously, but with the west and east entries. The frequency of vehicles will start at 900 vehicles per hour per entry for the first period, and will evolve to 600 for the last two periods.

In [4]:
import numpy as np

period_time = 300

load_vector = np.array([900, 600, 600]) # Load vector : each value represents a frequency for a period, for all the network

coeff_matrix = np.array([
    # P1      P2      P3
    [0.0833, 0.1666, 0.0001], # Proportion of frequency for the route North / East
    [0.0833, 0.1666, 0.0001], # Proportion of frequency for the route North / South
    [0.0833, 0.1666, 0.0001], # Proportion of frequency for the route North / West
    [0.0833, 0.0001, 0.1666], # Proportion of frequency for the route East / North
    [0.0833, 0.0001, 0.1666], # Proportion of frequency for the route East / South
    [0.0833, 0.0001, 0.1666], # Proportion of frequency for the route East / West
    [0.0833, 0.1666, 0.0001], # Proportion of frequency for the route South / North
    [0.0833, 0.1666, 0.0001], # Proportion of frequency for the route South / East
    [0.0833, 0.1666, 0.0001], # Proportion of frequency for the route South / West
    [0.0833, 0.0001, 0.1666], # Proportion of frequency for the route West / North
    [0.0833, 0.0001, 0.1666], # Proportion of frequency for the route West / East
    [0.0833, 0.0001, 0.1666], # Proportion of frequency for the route West / South
])

infrastructures = network.generate_infrastructures(
    lane_length=200,
    green_time=60,
    yellow_time=3,
    max_speed=50
)

flows = network.generate_flows_with_matrix(
    period_time=period_time,
    load_vector=load_vector,
    coeff_matrix=coeff_matrix,
    distribution='binomial'
)

exp_matrix = Experiment(
    name = 'matrix',
    infrastructures = infrastructures,
    flows = flows
)

exp_matrix.run(
    simulation_duration=(period_time*3),
    gui=True
)

exp_matrix.clean_files()

Success.
Success.


***Note :** SUMO doesn't allow to modify flows when they are defined. To solve this problem, sumo-experiments generates flows for each period, that start at the beginning and last until the the end of the period. Although there are 12 routes in the previous example, sumo-experiments generates actually 36 routes for the entire simulation.*

***Note :** As Sumo doesn't accept to create flows with a number of vehicles set to 0, we have to set the flows to a low frequency to make them unused. This flows can then generate a car sometimes.*

## Generate flows with a CSV file

We saw previously how to define variable flows with matrices, with numpy.array objects. This allows to generate matrices and use it. But, in some case, you may want to save persistent flow data, in a CSV file for example, and use it in different experiments. In order to use persistant data, sumo-experiments implements a function `import_flows_parameters_from_csv` that read a CSV file with flow data, and returns a load vector and a coefficient matrix. The function is located in the `util` subpackage.

The CSV file must be in that shape :
- The first line is the load vector, of size $x$.
- The rest of the file is the coeff matrix, of shape ($y$, $x$).
- No other data in the file.

The default separator for the data is the comma, but another one can be set using the function.

In [5]:
from sumo_experiments.util import import_flows_parameters_from_csv

load_vector = np.array([300, 200, 200]) # Load vector : each value represents a frequency for a period

coeff_matrix = np.array([
    # P1      P2      P3
    [0.0833, 0.1666, 0.0001], # Proportion of frequency for the route North / East
    [0.0833, 0.1666, 0.0001], # Proportion of frequency for the route North / South
    [0.0833, 0.1666, 0.0001], # Proportion of frequency for the route North / West
    [0.0833, 0.0001, 0.1666], # Proportion of frequency for the route East / North
    [0.0833, 0.0001, 0.1666], # Proportion of frequency for the route East / South
    [0.0833, 0.0001, 0.1666], # Proportion of frequency for the route East / West
    [0.0833, 0.1666, 0.0001], # Proportion of frequency for the route South / North
    [0.0833, 0.1666, 0.0001], # Proportion of frequency for the route South / East
    [0.0833, 0.1666, 0.0001], # Proportion of frequency for the route South / West
    [0.0833, 0.0001, 0.1666], # Proportion of frequency for the route West / North
    [0.0833, 0.0001, 0.1666], # Proportion of frequency for the route West / East
    [0.0833, 0.0001, 0.1666], # Proportion of frequency for the route West / South
])

to_csv = np.vstack((load_vector, coeff_matrix))
np.savetxt("tuto_4.csv", to_csv, delimiter=",")

load_vector_2, coeff_matrix_2 = import_flows_parameters_from_csv('tuto_4.csv')

print((load_vector == load_vector_2).all())
print((coeff_matrix == coeff_matrix_2).all())

True
True


The load vector and the coefficient matrix extracted from the `import_flows_parameters_from_csv` can then be used to custom an experiment.

In [6]:
period_time = 300

flows = network.generate_flows_with_matrix(
    period_time=period_time,
    load_vector=load_vector_2,
    coeff_matrix=coeff_matrix_2
)

exp_matrix = Experiment(
    name = 'matrix_2',
    infrastructures = infrastructures,
    flows = flows
)

exp_matrix.run(
    simulation_duration=(period_time*3),
    gui=True
)

exp_matrix.clean_files()

Success.
Success.


In [7]:
!rm tuto_4.csv

**You know now how to create simulations with variable flows. This allows you to create traffic scenarios to test traffic management systems in different conditions. However, this method can only be used with pre-set networks. If you want to reproduce this behaviour with a custom network, you must redefine a similar function, specific to this network. Custom networks are the subject of the next tutorial.**